In [6]:
import sys
import math

import numpy as np
import pandas as pd
from configparser import ConfigParser
import pandas.io.sql as psql
import psycopg2, psycopg2.extras
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource

sys.path.append('/home/adam/.pyenv/ep-chan/lib/python3.5/site-packages')

output_notebook()

Loading BokehJS ...

In [2]:
# db.py
def connect(params):
    """ Connect to the PostgreSQL database server """
    conn = None

    # connect to the PostgreSQL server
    conn = psycopg2.connect(**params)

    # create a cursor
    cur = conn.cursor()

    # execute a statement
    print('PostgreSQL database version:')
    cur.execute('SELECT version()')

    # display the PostgreSQL database server version
    db_version = cur.fetchone()
    print(db_version)

    # close the communication with the PostgreSQL
    cur.close()

    return conn

def close(conn):
    if conn is not None:
        conn.close()
        print('Database connection closed.')

    else:
        print('conn is None')

def fetchRates(conn, min_date, max_date, coins):
    """ query rates """
    if conn == None:
        raise Exception("conn is required")

    q = []
    for coin in coins:
        q.append("""AVG(CASE WHEN UPPER(symbol) = '{}' THEN open ELSE NULL END) AS "{}" """.format(coin, coin))

    return psql.read_sql("""
        SELECT
            date::timestamptz::date AS "date",
            {}

        FROM
            coinmarketcap_5yrs

        WHERE
            date IS NOT NULL AND
            date >= %s AND
            date <= %s AND
            UPPER("symbol") = ANY(%s)

        GROUP BY
            date::timestamptz

        ORDER BY
            date ASC
    """.format(", ".join(q)), conn, index_col="date", parse_dates=["date"], coerce_float=True, params=(min_date, max_date, coins.tolist()),)

In [3]:
# config.py
def configDB(filename='config.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)

        for param in params:
            db[param[0]] = param[1]

    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

def configSettings(filename='config.ini', section='settings'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to settings
    settings = {}
    if parser.has_section(section):
        params = parser.items(section)

        for param in params:
            settings[param[0]] = param[1]

    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return settings

In [32]:
# utils.py
# note: https://quant.stackexchange.com/questions/39839/how-to-calculate-sharpe-ratio-from-returns
def calcSR(trades, rates, coins, percent_aum):
    # calculate the cumulative returns
    # first, calculate the daily portfolio value
    starting_principle = 1
    open_positions = {}
    daily_account_value = {}
    opened_trades = pd.DataFrame().reindex_like(trades)
    opened_trades["multiplier"] = None

    for coin in coins:
        open_positions[coin] = {"history": [[], []], "outstanding_shares": 0.0} # history is price, daily purchases / sales

    for idx1, rates_row in rates.iterrows():
        # open new trades; the index (idx1) is timestamp open
        try:
            days_trades = pd.DataFrame(trades.loc[trades["timestamp open"] == idx1.date().isoformat()])
            if not days_trades.empty:
                for _, trade in days_trades.iterrows():
                    portfolio_value = calculate_portfolio_value(starting_principle, open_positions, rates_row)
                    cost = calculate_cost(trade)
                    multiplier = float(portfolio_value) * float(percent_aum) / float(cost)

                    if trade["side"].upper() == "BUY":
                        open_positions[trade["p"]]["history"][0].append(trade["p rate open"])
                        open_positions[trade["p"]]["history"][1].append(multiplier * 1.0)
                        open_positions[trade["p"]]["outstanding_shares"] += multiplier * 1.0

                        open_positions[trade["q"]]["history"][0].append(trade["q rate open"])
                        open_positions[trade["q"]]["history"][1].append(-1.0 * multiplier * trade["hedge ratio"])
                        open_positions[trade["q"]]["outstanding_shares"] -= multiplier * trade["hedge ratio"]

                        trade["multiplier"] = multiplier
                        opened_trades = opened_trades.append(trade, ignore_index=False)

                    else:
                        open_positions[trade["p"]]["history"][0].append(trade["p rate open"])
                        open_positions[trade["p"]]["history"][1].append(multiplier * -1.0)
                        open_positions[trade["p"]]["outstanding_shares"] -= multiplier * 1.0

                        open_positions[trade["q"]]["history"][0].append(trade["q rate open"])
                        open_positions[trade["q"]]["history"][1].append(multiplier * trade["hedge_ratio"])
                        open_positions[trade["q"]]["outstanding_shares"] += multiplier * trade["hedge ratio"]

                        trade["multiplier"] = multiplier
                        opened_trades = opened_trades.append(trade, ignore_index=False)

        except (Exception) as error:
            # exc_type, exc_obj, exc_tb = sys.exc_info()
            # print("open trades exception {} at {}".format(error, exc_tb.tb_lineno))
            # fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            # print(exc_type, fname, )
            # do nothing
            _ = 1

        # close trades
        try:
            close_trades = pd.DataFrame(opened_trades.loc[trades["timestamp close"] == idx1.date().isoformat()])
            if not close_trades.empty:
                for _, trade in close_trades.iterrows():
                    if trade["side"].upper() == "BUY":
                        open_positions[trade["p"]]["history"][0].append(trade["p rate close"])
                        open_positions[trade["p"]]["history"][1].append(-1.0 * trade["multiplier"])
                        open_positions[trade["p"]]["outstanding_shares"] -= 1.0 * trade["multiplier"]

                        open_positions[trade["q"]]["history"][0].append(trade["q rate close"])
                        open_positions[trade["q"]]["history"][1].append(trade["multiplier"] * trade["hedge ratio"])
                        open_positions[trade["q"]]["outstanding_shares"] += trade["multiplier"] * trade["hedge ratio"]

                    else:
                        open_positions[trade["p"]]["history"][0].append(trade["p rate close"])
                        open_positions[trade["p"]]["history"][1].append(1.0 * trade["multiplier"])
                        open_positions[trade["p"]]["outstanding_shares"] += 1.0 * trade["multiplier"]

                        open_positions[trade["q"]]["history"][0].append(trade["q rate close"])
                        open_positions[trade["q"]]["history"][1].append(-1 * trade["multiplier"] * trade["hedge ratio"])
                        open_positions[trade["q"]]["outstanding_shares"] -= trade["multiplier"] * trade["hedge ratio"]

        except (Exception) as error:
            # exc_type, exc_obj, exc_tb = sys.exc_info()
            # print("close trades exception {} at {}".format(error, exc_tb.tb_lineno))
            # do nothing
            _ = 1

        # calculate the performance
        portfolio_value = calculate_portfolio_value(starting_principle, open_positions, rates_row)

        # update the daily account value dict
        daily_account_value[idx1.date().isoformat()] = portfolio_value

    # turn the daily account val dict into a pd DataFrame
    daily_balances = pd.DataFrame.from_dict(daily_account_value,
                                            orient="index",
                                            columns=["balance"])
    daily_balances.index = pd.to_datetime(daily_balances.index)
    daily_balances.sort_index(inplace=True)
    daily_balances = daily_balances.fillna(method='backfill')
    # print(daily_balances)

    # calculate the cumulative daily returns in $
    # R = pd.DataFrame(daily_balances.diff()).cumsum()
    # print(R)

    # now calculate the percent cumulative daily returns
    # Approach 1
    # r = (R - R.shift(1))/R.shift(1)
    r = (daily_balances - daily_balances.shift(1))/daily_balances.shift(1)
    r["balance"] = r["balance"].fillna(method='pad')
    r = r.apply(lambda x: pd.Series(x.dropna().values))[1:]
    # r = remove_outlier(r, "balance")
    # print(r)

    # Approach 2
    # r = R.diff()

    # finally, return the sharpe ratio. note: crypto is traded 24/7/365 so no need for sqrt(252)
    print("Final aum {}".format(daily_balances[-1:]["balance"]))
    print("mean: {}; std dev: {}".format(r["balance"].mean(), r["balance"].std()))
    apr = (daily_balances["balance"].tail(1).iloc[0] / daily_balances["balance"].head(1).iloc[0]) ** (365.0 / float(len(daily_balances.index))) - 1.0
    return math.sqrt(365) * (r["balance"].mean() / r["balance"].std()), apr, daily_balances

def calculate_portfolio_value(starting_principle, open_positions, rates_row):
    portfolio_value = 0.0
    for coin in open_positions:
        weighted_cap = sum([i*j for i,j in zip(open_positions[coin]["history"][0],open_positions[coin]["history"][1])]) # this is just a sum product, even though it looks weird...
        current_cap = open_positions[coin]["outstanding_shares"] * rates_row[coin]

        portfolio_value += (current_cap - weighted_cap)

    return starting_principle + portfolio_value

def calculate_cost(trade):
    return trade["p rate open"] + trade["hedge ratio"] * trade["q rate open"]

#------------------------------------------------------------------------------
# accept a dataframe, remove outliers, return cleaned data in a new dataframe
# see http://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm
#------------------------------------------------------------------------------
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]

    return df_out

In [31]:
print('Loading configs')
# load the configs
dbConfig = configDB()
settingsConfig = configSettings()

# connect to the db
print('Connecting to the PostgreSQL database.')
conn = connect(dbConfig)

# load the trades file
print('Loading trades file')
# trades = pd.read_csv(settingsConfig["trades"], sep='\t', index_col="timestamp open", parse_dates=["timestamp open", "timestamp close"], infer_datetime_format=True)
trades = pd.read_csv(settingsConfig["trades"], sep='\t', parse_dates=["timestamp open", "timestamp close"], infer_datetime_format=True)
trades.sort_index(inplace=True)

# reasign the trades to the period of interes
mask = (trades['timestamp open'] > settingsConfig["start_date"]) & (trades['timestamp open'] <= settingsConfig["end_date"])
trades = trades.loc[mask]

# get the unique coins
coins = np.unique(list(trades["p"]) + list(trades["q"]))
print(coins)

# fetch the rates
print('Fetching rates')
rates = fetchRates(conn, settingsConfig["start_date"], settingsConfig["end_date"], coins)
rates = rates.fillna(method='backfill')
close(conn)

# calc sharpe ratio
print('Calculating Sharpe Ratio')
sr, apr, portfolio_balance = calcSR(trades, rates, coins, settingsConfig["ratio_aum"])

# display to user
print("Sharpe Ratio is {:.2f}; APR is {:.2f}".format(sr, apr))

Loading configs
Connecting to the PostgreSQL database.
PostgreSQL database version:
('PostgreSQL 9.6.11 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',)
Loading trades file
['AION' 'BAT' 'BTC' 'BTG' 'DASH' 'EDO' 'ELF' 'EOS' 'ETC' 'ETH' 'FUN' 'GNT'
 'IOST' 'LTC' 'MANA' 'NEO' 'OMG' 'QASH' 'QTUM' 'REP' 'SAN' 'SNT' 'TRX'
 'XLM' 'XMR' 'XRP' 'XVG' 'ZEC' 'ZRX']
Fetching rates
Database connection closed.
Calculating Sharpe Ratio
Final aum 2018-12-31    1.781324
Name: balance, dtype: float64
mean: 0.001693098681627693; std dev: 0.014279755574901777
APR:  1.7813239525991462
Sharpe Ratio is 2.27; APR is 0.78


In [15]:
#data = ColumnDataSource(portfolio_balance) # create object for bokeh plot
p = figure(x_axis_type='datetime', plot_width=900, plot_height=600)
p.line(x = portfolio_balance.index.values, y = portfolio_balance['balance'], line_width=2)
show(p)